# Covid-19: From model prediction to model predictive control

## JPGMs tryout of the QALY calculation

*Original code by Juan Pablo. Modified by T.W. Alleman in consultation with the BIOMATH research unit headed by prof. Ingmar Nopens.*

Copyright (c) 2020 by BIOMATH, Ghent University. All Rights Reserved.

In `src/covid19model/models.py`, two national-level extended SEIRD compartmental models are coded,
1. COVID19_SEIRD: a deterministic model, using ordinary differential equations (ODEs). The equations are integrated using Scipy's `odeint` with a variable timestep, it is a 'continuous-time' model.
2. COVID19_SEIRD_sto: a stochastic model, using stochastic difference equations (SDEs). The equations are iterated using a fixed timestep of one day and is said to be a 'discrete-time' model.

The deterministic model was used for all results detailed in our first COVID-19 related preprint: *A deterministic, age-stratified, extended SEIRD model for assessing the effect of non-pharmaceutical interventions on SARS-CoV-2 spread in Belgium*, which can be found here: https://www.medrxiv.org/content/10.1101/2020.07.17.20156034v2

The documentation given in this tutorial has been redrafted from the above preprint. References have been omitted for the sake of simplicity.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm
import pandas as pd
import datetime
import scipy
import json
import random

Load the covid 19 custom development code

In [ ]:
from covid19model.optimization import objective_fcns
from covid19model.models import models
from covid19model.data import google, sciensano, polymod, model_parameters
from covid19model.visualization.output import population_status, infected
from covid19model.visualization.optimization import plot_fit, traceplot

In [ ]:
# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

## Model initialization

In [ ]:
# Load the interaction matrices (size: 9x9)
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = polymod.get_interaction_matrices()
# Define the number of age categories
levels = initN.size

In [ ]:
# Load the compliance model
from covid19model.models.compliance import ramp_2

In [ ]:
# Load the model parameters using `get_COVID19_SEIRD_parameters()`.
params = model_parameters.get_COVID19_SEIRD_parameters()
# Add the delayed ramp parameters to the parameter dictionary to use compliance.
params.update({'l': 1,
              'tau': 5})
# Define the initial condition: one exposed inidividual in every age category
initial_states = {'S': initN, 'E': np.ones(levels)}
# Initialize the model
model = models.COVID19_SEIRD(initial_states, params, compliance=ramp_2)

In [ ]:
# Load the dictionary containing the posterior parameter distributions obtained from calibrating the model to Belgian hospitalization data
with open('../../data/interim/model_parameters/deterministic_22072020.json', 'r') as fp:
    samples_dict = json.load(fp)

lag_time = samples_dict['lag_time']
model.extraTime = samples_dict['lag_time']
model.parameters['beta'] = np.mean(samples_dict['beta'])
model.parameters['l'] = np.mean(samples_dict['l'])
model.parameters['tau'] = np.mean(samples_dict['tau'])
prevention = np.mean(samples_dict['prevention'])

## Simulate the model

In [ ]:
sim_time=150
out=model.sim(sim_time)

## QALY calculations

In [ ]:
from covid19model.models.QALY import create_life_table

#### Quality of life parameters

In [ ]:
# Standarized Mortality ratio. It is used to adjust for increased mortality 
#due to comorbidities. It modifies l(x), the term in life tables that 
# corresponds to the number of people per 100000 hab that survives to age x
SMR=1 
#Adjustment paramter to account for additional impact on quality of life 
#due to comorbidity
qCM=1
#Disccount rate. It is included due to the time preference. Things in the present
#are valued more than in the future. 5% is standard practice
r=0.05

##Import input data##
#Belgian life table information
#x: AGe
#q_x: Probability that someone aged exactly x will die before reaching age x+1
input_life_table=pd.read_csv('../../data/raw/QALYs/Life_table_Belgium_2018.csv',sep=';')
#Belgian EQ-5D QoL Survey data
input_QoL=pd.read_csv('../../data/raw/QALYs/QoL_scores_Belgium_2018_v3.csv',sep=';')

#Deaths per age group dataframe initialization
deaths_input=pd.DataFrame({'age_group':['0-9','10-19','20-29','30-39','40-59','50-59','60-69','70-79','80+'],
                          'group_limit':[9,19,29,39,49,59,69,89,110],
                          'deaths':[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]})

In [ ]:
#Create life Belgium's life table
life_table_1=create_life_table(input_life_table,input_QoL,SMR,qCM,r)
#Extraact death information
total_deaths=out['D']

low_limit=0
#Calculate average number of lost QALYs per person for the available age groups
for i in range(len(deaths_input['group_limit'])):
    avg_dQALY=np.mean(life_table_1.loc[low_limit:deaths_input.loc[i,'group_limit'],'QALY_x'])
    deaths_input.loc[i,'lost_QALY_pp']=avg_dQALY
    low_limit=deaths_input.loc[i,'group_limit']+1

#Initialize output array
lost_QALY=np.zeros((sim_time,len(deaths_input['group_limit'])))

#Calculate total number of lost QALYs for each age group for each time step                   
for j in range(sim_time):
    deaths_time=deaths_input.copy()
    for a in range(len(deaths_time['group_limit'])):
        
        deaths_time.loc[a,'deaths']=np.array(total_deaths[a][j])
        deaths_time['lost_QALY']=deaths_time['deaths']*deaths_time['lost_QALY_pp']
        lost_QALY[j,:]=deaths_time['lost_QALY']

In [ ]:
#Plot
Palette=cm.get_cmap('tab10_r', len(lost_QALY[j,:])).colors


fig, ax = plt.subplots(figsize=(20,12))

for i in range(len(lost_QALY[j,:])):
               
    ax.plot(lost_QALY[:,i],linewidth=3,label=deaths_input['age_group'][i],color=Palette[i])
    
#ax.plot(lost_QALY.sum(axis=1),color='k',linewidth=4,label='Total')

ax.set_xlim(left=50,right=sim_time)
ax.set_ylim(bottom=0)
ax.set_xlabel('Time',fontsize=25)
ax.set_ylabel('Lost QALYs',fontsize=25)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(labelsize=25)
ax.spines["bottom"].set_linewidth(2)
ax.spines["left"].set_linewidth(2)
ax.set_axisbelow(True)
ax.legend(loc='best', fancybox=True, frameon=True, framealpha=1, fontsize=15,title='Age Group')  
